[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션 ID 설정하여, 대화 진행합니다. (muti-turn conversation)
LLM 모델이 과거 대화를 알지 (기억) 못하는 상황을 만드세요.

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, trim_messages
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from operator import itemgetter

load_dotenv()

True

In [5]:
model = ChatOpenAI(model= 'gpt-4o')

trimmer = trim_messages(
    max_tokens = 100,
    strategy = 'last',
    token_counter = model,
    include_system = True,
    allow_particle = False,
    # start_on = 'human'
)

In [6]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [7]:
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

In [8]:
chain = (RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
         | prompt
         | model)

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages",
    history_messages_key= "chat_history"
)

In [13]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="육아휴직 며칠 쉴 수 있어요?")],
        "query": "육아휴직 며칠 쉴 수 있어요?",
        "law": "근로기준법 제74조"
    },
    config={"configurable": {"session_id": "abc123"}}
)

TypeError: trim_messages() got an unexpected keyword argument 'allow_particle'

In [10]:
print(response.content)

NameError: name 'response' is not defined